In [14]:
from pathlib import Path
from os import listdir
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Files to analyse 

In [15]:
source_folder = Path(r"E:\Post-doc_LMDC_Paendora2\final_work\correct_analysis\with_complete_dpe_data_by_roof")

In [16]:
dan_bound_path = r"E:\Post-doc_LMDC_Paendora2\final_work\correct_analysis\Danube_with_U_bound_association_by_roof.xlsx"
dan = pd.read_excel(dan_bound_path, index_col=0)
dan.head()

,USAGE,NOM_TYPOLOGIE,PERIODE,TERRITOIRE,DISPOSITIF_M1,PORTEUR_M1,EP_PORTEUR_M1,ISOLANT_M1,EP_ISOLANT_M1,REVETEMENT_INT_M1,...,Uph_0_moy,Uph_min,Uph_max,Uw_0,Uw_min,Uw_max,archetype,one_one,two_one,fichier
0,BATIMENT AGRICOLE,BA,P1,FRANCE,PAV_M_P1_TBr_br,BRIQUE PLEINE,60,NON,0,ENDUIT,...,1.7375,1.2,2.5,1.35,1.35,2.5,P1-BA-BATIMENT AGRICOLE-FRANCE,NaN,NaN,NaN
1,BATIMENT AGRICOLE,BA,P1,FRANCE_BOIS_ARDOISE,PAV_M_P1_TBo-bo,BOIS,50,IR,0,NON,...,1.7375,1.2,2.5,0.80,0.80,2.5,P1-BA-BATIMENT AGRICOLE-FRANCE_BOIS_ARDOISE,NaN,NaN,NaN
2,BATIMENT AGRICOLE,BA,P1,FRANCE_BOIS_TUILE,PAV_M_P1_TBo-bo,BOIS,50,IR,0,NON,...,1.7375,1.2,2.5,0.80,0.80,2.5,P1-BA-BATIMENT AGRICOLE-FRANCE_BOIS_TUILE,NaN,NaN,NaN
3,BATIMENT AGRICOLE,BA,P1,FRANCE_BRIQUE_TUILE,PAV_M_P1_TBr_br,BRIQUE PLEINE,60,NON,0,ENDUIT,...,1.7375,1.2,2.5,1.35,1.35,2.5,P1-BA-BATIMENT AGRICOLE-FRANCE_BRIQUE_TUILE,NaN,NaN,NaN
4,BATIMENT AGRICOLE,BA,P1,FRANCE_PIERRE_CALCAIRE_ARDOISE,PAV_M_P1_TPi_Cal,CALCAIRE,80,NON,0,NON,...,1.7375,1.2,2.5,1.50,1.50,2.5,P1-BA-BATIMENT AGRICOLE-FRANCE_PIERRE_CALCAIRE...,NaN,NaN,NaN


In [17]:
files_anlz = list(dan[dan['fichier'].notnull()].fichier.unique())
print(len(files_anlz))
files_anlz[:3]

34


['dpe_all3-len_916-P1-I-COMMERCE-FRANCE-nati.csv',
 'dpe_all3-len_1010-P1-I-HABITAT-FRANCE_BOIS_TUILE-uniq.csv',
 'dpe_all3-len_9405-P1-I-HABITAT-FRANCE_BRIQUE-toit.csv']

#### order by period


In [18]:
import re

def get_per(value):
    match = re.search(r'(?<=-P)\w+', value)
    return int(match.group(0))

files_anlzs_sorted = sorted(files_anlz, key=get_per)

print(files_anlzs_sorted)

['dpe_all3-len_916-P1-I-COMMERCE-FRANCE-nati.csv', 'dpe_all3-len_1010-P1-I-HABITAT-FRANCE_BOIS_TUILE-uniq.csv', 'dpe_all3-len_9405-P1-I-HABITAT-FRANCE_BRIQUE-toit.csv', 'dpe_all3-len_13788-P1-I-HABITAT-FRANCE_PIERRE_CALCAIRE-toit.csv', 'dpe_all3-len_621-P1-I-HABITAT-FRANCE_PIERRE_GNEISS_TUILE-uniq.csv', 'dpe_all3-len_1961-P1-I-HABITAT-FRANCE_PIERRE_GRANITE-toit.csv', 'dpe_all3-len_5661-P1-I-HABITAT-FRANCE_PIERRE_GRES-toit.csv', 'dpe_all3-len_1594-P1-I-HABITAT-FRANCE_PIERRE_MEULIERE_TUILE-uniq.csv', 'dpe_all3-len_617-P1-I-HABITAT-FRANCE_PIERRE_SCHISTE_TUILE-uniq.csv', 'dpe_all3-len_1074-P1-I-HABITAT-FRANCE_TERRE_TUILE-uniq.csv', 'dpe_all3-len_2141-P1-P-HABITAT-FRANCE_BOIS_TUILE-uniq.csv', 'dpe_all3-len_34181-P1-P-HABITAT-FRANCE_BRIQUE-toit.csv', 'dpe_all3-len_27244-P1-P-HABITAT-FRANCE_PIERRE_CALCAIRE-toit.csv', 'dpe_all3-len_1637-P1-P-HABITAT-FRANCE_PIERRE_GNEISS_TUILE-uniq.csv', 'dpe_all3-len_5265-P1-P-HABITAT-FRANCE_PIERRE_GRANITE-toit.csv', 'dpe_all3-len_13447-P1-P-HABITAT-FRANCE_PIE

### Position isolant

In [19]:
def position_isolant_mur(dpe, root_name):
    
    u_median = dpe[['adedpe202006_logtype_mur_pos_isol_ext',
                'adedpe202006_logtype_mur_u_ext']].groupby(['adedpe202006_logtype_mur_pos_isol_ext']).median()
    pourcentage_type = (dpe["adedpe202006_logtype_mur_pos_isol_ext"].value_counts() *100 /
                                dpe.adedpe202006_logtype_mur_pos_isol_ext.notnull().sum())

    pos_is = pd.concat([pourcentage_type, u_median],axis=1).sort_values("adedpe202006_logtype_mur_pos_isol_ext", ascending=False)
    pos_is = pos_is.T.round(2)
    # add multiindex
    index_test = [np.array([f'{root_name}', f'{root_name}']),
                 np.array(['Pourcentage (%)', 'U median (W/m²/K)', ])]
    pos_is.index = index_test
    
    return pos_is

In [20]:
def position_isolant_toit(dpe, root_name):
    u_median = dpe[['adedpe202006_logtype_ph_pos_isol',
                'adedpe202006_logtype_ph_u']].groupby(['adedpe202006_logtype_ph_pos_isol']).median()
    pourcentage_type = (dpe["adedpe202006_logtype_ph_pos_isol"].value_counts() *100 /
                                dpe.adedpe202006_logtype_ph_pos_isol.notnull().sum())

    pos_is = pd.concat([pourcentage_type, u_median],axis=1).sort_values("adedpe202006_logtype_ph_pos_isol", ascending=False)
    pos_is = pos_is.T.round(2)
    # add multiindex
    index_test = [np.array([f'{root_name}', f'{root_name}']),
                 np.array(['Pourcentage (%)', 'U median (W/m²/K)', ])]
    pos_is.index = index_test
    
    return pos_is

#### Run function to all

In [21]:
pos_isol_murs = []
pos_isol_toits = []

for csv in files_anlzs_sorted:   
    path = source_folder / csv
    root_name = "-".join(path.stem.split("-")[2:-1]) 
    print("\n", root_name)
    print(csv)
    dpe = pd.read_csv(path)
    
    pos_is_mur = position_isolant_mur(dpe, root_name)
    pos_isol_murs.append(pos_is_mur)

    pos_is_toit = position_isolant_toit(dpe, root_name)
    pos_isol_toits.append(pos_is_toit)


 P1-I-COMMERCE-FRANCE
dpe_all3-len_916-P1-I-COMMERCE-FRANCE-nati.csv

 P1-I-HABITAT-FRANCE_BOIS_TUILE
dpe_all3-len_1010-P1-I-HABITAT-FRANCE_BOIS_TUILE-uniq.csv

 P1-I-HABITAT-FRANCE_BRIQUE
dpe_all3-len_9405-P1-I-HABITAT-FRANCE_BRIQUE-toit.csv


C:\Users\ldecarva\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (155,172,299,301) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)



 P1-I-HABITAT-FRANCE_PIERRE_CALCAIRE
dpe_all3-len_13788-P1-I-HABITAT-FRANCE_PIERRE_CALCAIRE-toit.csv


C:\Users\ldecarva\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (172) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)



 P1-I-HABITAT-FRANCE_PIERRE_GNEISS_TUILE
dpe_all3-len_621-P1-I-HABITAT-FRANCE_PIERRE_GNEISS_TUILE-uniq.csv

 P1-I-HABITAT-FRANCE_PIERRE_GRANITE
dpe_all3-len_1961-P1-I-HABITAT-FRANCE_PIERRE_GRANITE-toit.csv

 P1-I-HABITAT-FRANCE_PIERRE_GRES
dpe_all3-len_5661-P1-I-HABITAT-FRANCE_PIERRE_GRES-toit.csv


...

C:\Users\ldecarva\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8,9,37,50,51,52,53,54,55,56,57,75,127,128,130,172,199,205,222,248,250,251,299,301,302,303,304) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)



 P7-I-HABITAT-FRANCE
dpe_all3-len_9137-P7-I-HABITAT-FRANCE-toit.csv


C:\Users\ldecarva\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (127,128,130,299,300,301) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)



 P7-P-HABITAT-FRANCE
dpe_all3-len_48162-P7-P-HABITAT-FRANCE-toit.csv


C:\Users\ldecarva\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (75,127,128,130,199,248,299,301,303,304) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#### Format wall isol output 

In [22]:
resume_isol_mur = pd.concat(pos_isol_murs) 
resume_isol_mur.head()

ITI  non isole   ITR  \
P1-I-COMMERCE-FRANCE           Pourcentage (%)    73.51      22.41  1.77   
                               U median (W/m²/K)   0.40       2.00  0.37   
P1-I-HABITAT-FRANCE_BOIS_TUILE Pourcentage (%)    72.97      20.62  1.60   
                               U median (W/m²/K)   0.36       2.00  0.36   
P1-I-HABITAT-FRANCE_BRIQUE     Pourcentage (%)    70.64      25.55  0.82   

                                                   ITE  ITI+ITE  ITI+ITR  \
P1-I-COMMERCE-FRANCE           Pourcentage (%)    1.10     0.66     0.44   
                               U median (W/m²/K)  0.42     0.22     0.34   
P1-I-HABITAT-FRANCE_BOIS_TUILE Pourcentage (%)    2.80     1.20     0.30   
                               U median (W/m²/K)  0.34     0.35     0.25   
P1-I-HABITAT-FRANCE_BRIQUE     Pourcentage (%)    1.76     0.73     0.40   

                                                   MOB  ITE+ITR  isole  
P1-I-COMMERCE-FRANCE           Pourcentage (%)    0.11      NaN    NaN  
                               U median (W/m²/K)  1.70      NaN    NaN  
P1-I-HABITAT-FRANCE_BOIS_TUILE Pourcentage (%)    0.50      NaN    NaN  
                               U median (W/m²/K)  0.45      NaN    NaN  
P1-I-HABITAT-FRANCE_BRIQUE     Pourcentage (%)    0.09     0.01    NaN

In [23]:
# check values to ignore
resume_isol_mur.mean().sort_values(ascending=False)

ITI          35.274559
non isole    13.074412
ITE           1.494853
ITR           0.857879
ITI+ITE       0.663788
ITI+ITR       0.465000
isole         0.433571
MOB           0.345370
ITE+ITR       0.231111
dtype: float64

In [24]:
cols_to_drop = ['ITI+ITE', 'ITI+ITR', 'MOB','ITE+ITR', 'isole',]
resume_isol_mur["autres"]  = resume_isol_mur[cols_to_drop].sum(axis=1)
resume_isol_mur = resume_isol_mur[['ITI', 'ITE', 'ITR',  'non isole','autres']]
resume_isol_mur.head()

ITI   ITE   ITR  \
P1-I-COMMERCE-FRANCE           Pourcentage (%)    73.51  1.10  1.77   
                               U median (W/m²/K)   0.40  0.42  0.37   
P1-I-HABITAT-FRANCE_BOIS_TUILE Pourcentage (%)    72.97  2.80  1.60   
                               U median (W/m²/K)   0.36  0.34  0.36   
P1-I-HABITAT-FRANCE_BRIQUE     Pourcentage (%)    70.64  1.76  0.82   

                                                  non isole  autres  
P1-I-COMMERCE-FRANCE           Pourcentage (%)        22.41    1.21  
                               U median (W/m²/K)       2.00    2.26  
P1-I-HABITAT-FRANCE_BOIS_TUILE Pourcentage (%)        20.62    2.00  
                               U median (W/m²/K)       2.00    1.05  
P1-I-HABITAT-FRANCE_BRIQUE     Pourcentage (%)        25.55    1.23

#### Format roof isol output 

In [25]:
resume_isol_toit = pd.concat(pos_isol_toits) 
resume_isol_toit.head()

ITI  non isole   ITE  \
P1-I-COMMERCE-FRANCE           Pourcentage (%)    71.23      25.52  2.44   
                               U median (W/m²/K)   0.20       2.50  0.30   
P1-I-HABITAT-FRANCE_BOIS_TUILE Pourcentage (%)    63.78      31.41  2.35   
                               U median (W/m²/K)   0.20       2.50  0.24   
P1-I-HABITAT-FRANCE_BRIQUE     Pourcentage (%)    61.14      35.07  2.60   

                                                  isole  ITE+ITI  ITI+ITR  
P1-I-COMMERCE-FRANCE           Pourcentage (%)     0.58     0.23      NaN  
                               U median (W/m²/K)   0.24     0.56      NaN  
P1-I-HABITAT-FRANCE_BOIS_TUILE Pourcentage (%)     2.14     0.32      NaN  
                               U median (W/m²/K)   0.24     0.20      NaN  
P1-I-HABITAT-FRANCE_BRIQUE     Pourcentage (%)     1.09     0.10      NaN

In [26]:
resume_isol_toit.mean().sort_values(ascending=False)

ITI          34.505588
non isole    13.819706
ITE           2.460588
isole         0.736618
ITE+ITI       0.205152
ITI+ITR       0.115000
dtype: float64

In [27]:
cols_to_drop = ['isole','ITE+ITI', 'ITI+ITR',]
resume_isol_toit["autres"]  = resume_isol_toit[cols_to_drop].sum(axis=1)
resume_isol_toit = resume_isol_toit[['ITI', 'ITE', 'non isole','autres']]
resume_isol_toit.head()

ITI   ITE  non isole  \
P1-I-COMMERCE-FRANCE           Pourcentage (%)    71.23  2.44      25.52   
                               U median (W/m²/K)   0.20  0.30       2.50   
P1-I-HABITAT-FRANCE_BOIS_TUILE Pourcentage (%)    63.78  2.35      31.41   
                               U median (W/m²/K)   0.20  0.24       2.50   
P1-I-HABITAT-FRANCE_BRIQUE     Pourcentage (%)    61.14  2.60      35.07   

                                                  autres  
P1-I-COMMERCE-FRANCE           Pourcentage (%)      0.81  
                               U median (W/m²/K)    0.80  
P1-I-HABITAT-FRANCE_BOIS_TUILE Pourcentage (%)      2.46  
                               U median (W/m²/K)    0.44  
P1-I-HABITAT-FRANCE_BRIQUE     Pourcentage (%)      1.19

#### Export tables

In [28]:
resume_isol_mur.to_excel("resume_pos_isolat_mur.xlsx")
resume_isol_toit.to_excel("resume_pos_isolat_toit.xlsx")

In [29]:
resume_isol_mur

ITI   ITE   ITR  \
P1-I-COMMERCE-FRANCE           Pourcentage (%)    73.51  1.10  1.77   
                               U median (W/m²/K)   0.40  0.42  0.37   
P1-I-HABITAT-FRANCE_BOIS_TUILE Pourcentage (%)    72.97  2.80  1.60   
                               U median (W/m²/K)   0.36  0.34  0.36   
P1-I-HABITAT-FRANCE_BRIQUE     Pourcentage (%)    70.64  1.76  0.82   
...                                                 ...   ...   ...   
P6-P-HABITAT-FRANCE            U median (W/m²/K)   0.36  0.35  0.35   
P7-I-HABITAT-FRANCE            Pourcentage (%)    75.41  7.24  1.94   
                               U median (W/m²/K)   0.36  0.30  0.34   
P7-P-HABITAT-FRANCE            Pourcentage (%)    85.73  2.01  2.03   
                               U median (W/m²/K)   0.35  0.26  0.25   

                                                  non isole  autres  
P1-I-COMMERCE-FRANCE           Pourcentage (%)        22.41    1.21  
                               U median (W/m²/K)       2.00    2.26  
P1-I-HABITAT-FRANCE_BOIS_TUILE Pourcentage (%)        20.62    2.00  
                               U median (W/m²/K)       2.00    1.05  
P1-I-HABITAT-FRANCE_BRIQUE     Pourcentage (%)        25.55    1.23  
...                                                     ...     ...  
P6-P-HABITAT-FRANCE            U median (W/m²/K)       0.47    1.52  
P7-I-HABITAT-FRANCE            Pourcentage (%)        10.13    5.28  
                               U median (W/m²/K)       1.90    1.52  
P7-P-HABITAT-FRANCE            Pourcentage (%)         5.51    4.72  
                               U median (W/m²/K)       1.90    1.13  

[68 rows x 5 columns]

In [30]:
resume_isol_toit

ITI   ITE  non isole  \
P1-I-COMMERCE-FRANCE           Pourcentage (%)    71.23  2.44      25.52   
                               U median (W/m²/K)   0.20  0.30       2.50   
P1-I-HABITAT-FRANCE_BOIS_TUILE Pourcentage (%)    63.78  2.35      31.41   
                               U median (W/m²/K)   0.20  0.24       2.50   
P1-I-HABITAT-FRANCE_BRIQUE     Pourcentage (%)    61.14  2.60      35.07   
...                                                 ...   ...        ...   
P6-P-HABITAT-FRANCE            U median (W/m²/K)   0.19  0.20       2.50   
P7-I-HABITAT-FRANCE            Pourcentage (%)    62.36  7.47      28.43   
                               U median (W/m²/K)   0.20  0.27       2.50   
P7-P-HABITAT-FRANCE            Pourcentage (%)    89.08  3.59       5.59   
                               U median (W/m²/K)   0.16  0.23       2.50   

                                                  autres  
P1-I-COMMERCE-FRANCE           Pourcentage (%)      0.81  
                               U median (W/m²/K)    0.80  
P1-I-HABITAT-FRANCE_BOIS_TUILE Pourcentage (%)      2.46  
                               U median (W/m²/K)    0.44  
P1-I-HABITAT-FRANCE_BRIQUE     Pourcentage (%)      1.19  
...                                                  ...  
P6-P-HABITAT-FRANCE            U median (W/m²/K)    0.61  
P7-I-HABITAT-FRANCE            Pourcentage (%)      1.74  
                               U median (W/m²/K)    0.46  
P7-P-HABITAT-FRANCE            Pourcentage (%)      1.74  
                               U median (W/m²/K)    0.39  

[68 rows x 4 columns]